# Full Chain Integration Test

This notebook tests the **complete CMPT chain** using AgentOrchestrator.

**What it tests:**
- Chain registration with `register_cmpt_chain()`
- Full pipeline execution: Context → Prioritization → Response
- Step-by-step execution with `ao.run_step()`
- DAG visualization
- Error handling across steps

In [ ]:
# Setup path
import sys
sys.path.insert(0, "../..")

from agentorchestrator import AgentOrchestrator
from cmpt import register_cmpt_chain
from cmpt.services import ChainRequest

## Test 1: Chain Registration

In [ ]:
def test_chain_registration():
    """Test that the CMPT chain registers correctly."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Verify chain exists
    assert "cmpt_chain" in ao.list_chains()
    
    # Verify steps exist
    steps = ao.list_steps()
    assert "context_builder" in steps
    assert "content_prioritization" in steps
    assert "response_builder" in steps
    
    print("✓ Chain registered: cmpt_chain")
    print(f"✓ Steps: {', '.join(steps)}")
    return ao

ao = test_chain_registration()

## Test 2: Chain Validation

In [ ]:
def test_chain_validation():
    """Test chain validation (like 'ao check')."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Validate chain
    ao.check("cmpt_chain")
    print("✓ Chain validation passed")
    return ao

ao = test_chain_validation()

## Test 3: DAG Visualization

In [ ]:
def test_dag_visualization():
    """Test DAG visualization."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    dag = ao.graph("cmpt_chain")
    print("✓ DAG Visualization:")
    print(dag)
    return ao

ao = test_dag_visualization()

## Test 4: Full Chain Execution

In [ ]:
async def test_full_chain_execution():
    """Test full chain execution."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Build request
    request = {
        "corporate_company_name": "Apple Inc",
        "meeting_datetime": "2025-02-15T10:00:00Z",
    }
    
    # Run chain
    result = await ao.launch("cmpt_chain", {"request": request})
    
    # Verify success
    assert result["success"] is True, f"Chain failed: {result.get('error')}"
    
    print(f"✓ Chain completed successfully")
    print(f"✓ Duration: {result.get('duration_ms', 0):.1f}ms")
    print(f"✓ Steps executed: {len(result.get('results', []))}")
    
    # Show step results
    print("\nStep Results:")
    for step_result in result.get("results", []):
        status = "✓" if step_result.get("success") else "✗"
        name = step_result.get("step", "unknown")
        duration = step_result.get("duration_ms", 0)
        print(f"  {status} {name}: {duration:.1f}ms")
    
    return result

result = await test_full_chain_execution()

## Test 5: Context Data Verification

In [ ]:
async def test_context_data():
    """Verify context data flows through the chain."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    result = await ao.launch("cmpt_chain", {
        "request": {
            "corporate_company_name": "Microsoft Corporation",
            "meeting_datetime": "2025-03-15T14:00:00Z",
        }
    })
    
    ctx_data = result.get("context", {}).get("data", {})
    
    # Verify context was populated
    assert ctx_data.get("company_name") == "Microsoft Corporation"
    assert ctx_data.get("ticker") is not None
    assert ctx_data.get("context_output") is not None
    assert ctx_data.get("prioritization_output") is not None
    
    print(f"✓ Company: {ctx_data.get('company_name')}")
    print(f"✓ Ticker: {ctx_data.get('ticker')}")
    print(f"✓ Context output captured")
    print(f"✓ Prioritization output captured")
    print(f"✓ Final response captured: {ctx_data.get('final_response') is not None}")
    
    return result

result = await test_context_data()

## Test 6: Individual Step Testing (NEW FEATURE)

In [ ]:
async def test_individual_step():
    """Test running individual steps with ao.run_step()."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Test Step 1: Context Builder
    print("Testing Step 1: context_builder")
    print("-" * 40)
    
    result = await ao.run_step("context_builder", {
        "request": {
            "corporate_company_name": "Tesla Inc",
            "meeting_datetime": "2025-04-01T09:00:00Z",
        }
    })
    
    assert result["success"] is True, f"Step failed: {result.get('error')}"
    print(f"  ✓ Success: {result['success']}")
    print(f"  ✓ Duration: {result['duration_ms']:.1f}ms")
    print(f"  ✓ Output keys: {list(result['output'].keys()) if result.get('output') else 'N/A'}")
    
    return result

result = await test_individual_step()

## Test 7: Step with Mock Dependencies

In [ ]:
async def test_step_with_mock_deps():
    """Test running a dependent step with mocked input."""
    from cmpt.services import ContextBuilderOutput, CompanyInfo, TemporalContext
    
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Create mock context output (normally from Step 1)
    mock_context_output = ContextBuilderOutput(
        company_info=CompanyInfo(
            name="Amazon",
            ticker="AMZN",
            industry="E-Commerce",
            sector="Consumer Discretionary",
        ),
        company_name="Amazon",
        ticker="AMZN",
        temporal_context=TemporalContext(
            meeting_date="2025-05-01",
            fiscal_quarter="2",
            fiscal_year="2025",
            days_to_earnings=15,
        ),
    )
    
    # Test Step 2: Content Prioritization (with mock input)
    print("Testing Step 2: content_prioritization (with mock input)")
    print("-" * 40)
    
    result = await ao.run_step("content_prioritization", {
        "context_output": mock_context_output,
    })
    
    assert result["success"] is True, f"Step failed: {result.get('error')}"
    print(f"  ✓ Success: {result['success']}")
    print(f"  ✓ Duration: {result['duration_ms']:.1f}ms")
    print(f"  ✓ Output keys: {list(result['output'].keys()) if result.get('output') else 'N/A'}")
    
    return result

result = await test_step_with_mock_deps()

## Test 8: Error Propagation

In [ ]:
async def test_error_propagation():
    """Test that errors are properly captured."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    # Try to run step 2 without required context (should fail gracefully)
    result = await ao.run_step("content_prioritization", {})
    
    # Should fail but not crash
    print(f"✓ Step handled missing dependencies gracefully")
    print(f"  Success: {result['success']}")
    if not result['success']:
        print(f"  Error: {result.get('error', {}).get('message', 'Unknown')}")
    
    return result

result = await test_error_propagation()

## Test 9: Multiple Companies

In [ ]:
async def test_multiple_companies():
    """Test chain with different companies."""
    ao = AgentOrchestrator(name="cmpt_test", isolated=True)
    register_cmpt_chain(ao)
    
    companies = [
        ("Apple Inc", "2025-02-01"),
        ("Google LLC", "2025-03-15"),
        ("Meta Platforms", "2025-04-20"),
    ]
    
    print("Running chain for multiple companies:")
    print("-" * 40)
    
    for company, date in companies:
        result = await ao.launch("cmpt_chain", {
            "request": {
                "corporate_company_name": company,
                "meeting_datetime": f"{date}T10:00:00Z",
            }
        })
        
        status = "✓" if result["success"] else "✗"
        duration = result.get("duration_ms", 0)
        print(f"  {status} {company}: {duration:.1f}ms")
    
    print("\n✓ All companies processed")

await test_multiple_companies()

## Summary

In [ ]:
print("="*60)
print("  Full Chain Integration Tests - All Passed!")
print("="*60)
print()
print("Tested:")
print("  ✓ Chain registration")
print("  ✓ Chain validation")
print("  ✓ DAG visualization")
print("  ✓ Full chain execution")
print("  ✓ Context data flow")
print("  ✓ Individual step testing (ao.run_step)")
print("  ✓ Step with mock dependencies")
print("  ✓ Error propagation")
print("  ✓ Multiple companies")
print()
print("The CMPT chain is fully functional with AgentOrchestrator!")